# Titanic - Machine Learning from Disaster

In this project, we will use use machine learning to create a model that predicts which passengers survived the Titanic shipwreck.

We will be working with the [Titanic Data Set from Kaggle](https://www.kaggle.com/c/titanic) and be using the Logistic Regression model in Python for classification.


The Version of the data set that we will be using is an "cleaner" version. We will also use use a "semi-cleaned" version of the titanic data set in order to save us time with the data cleaning.
## Import Libraries


In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline